In [1]:
import sys, os
sys.path.remove('C:\\Users\\Nathan\\Documents\\Dropbox\\Python')
sys.path.remove('C:\\users\\nathan\\documents\\Dropbox\\Python\\jpnotebooks\\Customer Apps\\Semma Data Scripts\\ C:\\program files\\python35\\Lib\\site-packages')
sys.path.append('C:\\Users\\Nathan\\Documents\\Dropbox\\Python\\scripts\\customer_apps\\multilogger\\src\\')
os.chdir("..\semma_demo")

## Imports from multilogger module

In [2]:
from multilogger import HelloMultiLogger, MLCallback

## Simple Example: Basic structure of generic callback routine receiving all events:

In [3]:
def callback(event, payload):
    if event == "ML_BEGIN_LOG":
        pass
    elif event == "ML_LOG_DATA":
        pass
    elif event == "ML_END_LOG":
        pass
    else:
        print("Received unknown event: %r"%event)

## Simple Example #2: A callback displaying the event name and payload

In [4]:
def my_callback(event, payload):
    print("Got event:", event)
    if event == "ML_BEGIN_LOG":
        print(*payload)
    elif event == "ML_LOG_DATA":
        print(*payload)
    elif event == "ML_END_LOG":
        pass
    else:
        print("Received unknown event: %r"%event)

## Simple CSV Writer: Using callbacks to open a file, log data received, and close file when done

In [5]:
def csv_writer(filename):
    f = None
    pvars = None
    
    def log(data):
        nonlocal f
        f.write(",".join(map(str, data))); f.write("\n")
    
    def csv_writer_cb(event, payload):
        nonlocal f, pvars
        
        if event == "ML_BEGIN_LOG":
            f = open(filename, 'w')
            
            # create pvars
            pvars = []
            for tag, vl in payload:
                for v in vl:
                    pvars.append(tag + "." + v)
            log(pvars)
        
        elif event == "ML_LOG_DATA":
            line = []
            for tag, vl in payload:
                for v in vl:
                    line.append(v)
            log(line)
        
        elif event == "ML_END_LOG":
            f.close()
        
        else:
            raise ValueError("Unknown event: %r"%event)
    
    return csv_writer_cb

## Simple CSV Writer #2: Class based callback handler.

Instead of using a callback function to process all events, the MLCallback class provides methods to shutter specific events to specific functions

In [10]:
class CSVWriter(MLCallback):
    def __init__(self, filename):
        super().__init__()
        if os.path.exists(filename):
            raise FileExistsError(filename)
        self._filename = filename
        self._f = None
        self._pvars = None
        
    def log_data(self, data):
        if self._f is None:
            raise ValueError("File not initialized")
        self._f.write(",".join(map(str,data)))
        self._f.write("\n")
        
    def onBeginLog(self, payload):
        
        # check file existance again when starting, just in case
        if os.path.exists(self._filename):
            raise FileExistsError(self._filename)
        
        self._f = open(self._filename, 'w')
        
        self._pvars = []
        for tag, vl in payload:
            for v in vl:
                self._pvars.append(tag + "." + v)
                
        self.log_data(self._pvars)
        
    def onDataReceived(self, payload):
        line = []
        for tag, vl in payload:
            for v in vl:
                line.append(v)
                
        self.log_data(line)
        
    def onEndLog(self, payload):
        self._f.close()
        self._f = None

## Example Usage of HelloMultiLogger

In [15]:
import time

# Choose all variables to log. All reactors will log the same variables.
vars_i_care_about = [
    'agitation.pv',
    'temperature.pv'
]

# Create logger 
ml = HelloMultiLogger(vars_i_care_about, 3)

ml.add_logger("192.168.1.12", "R&D1", 1)
ml.add_logger("192.168.1.16", "R&D2", 1)

ml.add_callback(my_callback)
ml.add_callback(csv_writer("test_multilogger.csv"))

# These two are equivalent
ml.add_callback(CSVWriter("test_multilogger2.csv"))
ml.add_callback(CSVWriter("test_multilogger3.csv").onEvent)

ml.start()
time.sleep(12)
ml.stop()

Got event: ML_BEGIN_LOG
('ML', ['Timestamp.Last', 'Elapsed Time.hr']) ('R&D1', ['Timestamp.Last', 'Elapsed Time.hr', 'agitation.pv', 'temperature.pv']) ('R&D2', ['Timestamp.Last', 'Elapsed Time.hr', 'agitation.pv', 'temperature.pv'])
Got event: ML_LOG_DATA
('ML', [datetime.datetime(2018, 10, 10, 16, 19, 45, 889896), 3.940138888888889e-05]) ('R&D1', [datetime.datetime(2018, 10, 10, 16, 19, 45, 876176), 3.752944444444444e-05, 0, 24.086410522460938]) ('R&D2', [datetime.datetime(2018, 10, 10, 16, 19, 45, 876937), 3.7464166666666666e-05, 0, 27.39652442932129])
Got event: ML_LOG_DATA
('ML', [datetime.datetime(2018, 10, 10, 16, 19, 48, 890225), 0.000872826111111111]) ('R&D1', [datetime.datetime(2018, 10, 10, 16, 19, 47, 978303), 0.000621453611111111, 0, 24.081539154052734]) ('R&D2', [datetime.datetime(2018, 10, 10, 16, 19, 47, 995255), 0.0006258858333333333, 0, 27.399036407470703])
Got event: ML_LOG_DATA
('ML', [datetime.datetime(2018, 10, 10, 16, 19, 51, 891750), 0.0017065830555555554]) ('R&

In [14]:
os.remove("test_multilogger.csv")
os.remove("test_multilogger2.csv")
os.remove("test_multilogger3.csv")

In [ ]:
%